In [ ]:
# Original script : https://github.com/Jiaqi1008/Emotion_detection/blob/master/ECG.py

import scipy.io as sio
import pandas as pd
from sklearn import preprocessing as pre
# Import the Neurokit2 library
import neurokit2 as nk # pip install https://github.com/neuropsychology/neurokit/zipball/master

In [ ]:
# Load the DREAMER dataset
path=u'DREAMER.mat'
data=sio.loadmat(path)

In [ ]:
# Create an empty dataframe ECG that will contain the features for all subjects and videos
ECG={}
# Create a counter to track the processing progress
total=0

In [ ]:
# Loop over the 23 subjects (indexed by k) and over the 18 videos (indexed by j)
print("ECG features are being extracted...")
for k in range(0,23):
        for j in range(0,18):
            # Load the data for the Baseline paradigm and for the Stimulus paradigm (Left)
            basl_l=data['DREAMER'][0,0]['Data'][0,k]['ECG'][0,0]['baseline'][0,0][j,0][:,0]
            stim_l=data['DREAMER'][0,0]['Data'][0,k]['ECG'][0,0]['stimuli'][0,0][j,0][:,0]
            # Load the data for the Baseline paradigm and for the Stimulus paradigm (Right)
            basl_r=data['DREAMER'][0,0]['Data'][0,k]['ECG'][0,0]['baseline'][0,0][j,0][:,1]
            stim_r=data['DREAMER'][0,0]['Data'][0,k]['ECG'][0,0]['stimuli'][0,0][j,0][:,1]
            # Use the Neurokit2 ecg_process command for each paradigm with a sampling rate of 256 Hz
            # Documentation : https://neurokit.readthedocs.io/en/latest/documentation.html#neurokit.ecg_process
            ecg_signals_b_l,info_b_l=nk.ecg_process(basl_l,sampling_rate=256)
            ecg_signals_s_l,info_s_l=nk.ecg_process(stim_l,sampling_rate=256)
            ecg_signals_b_r,info_b_r=nk.ecg_process(basl_r,sampling_rate=256)
            ecg_signals_s_r,info_s_r=nk.ecg_process(stim_r,sampling_rate=256)
            # Use the Neurokit2 ecg_intervalrelated command to generate a dataframe of the features for Left and Right
            # Documentation : https://neurokit2.readthedocs.io/en/latest/examples/intervalrelated.html
            processed_ecg_l=nk.ecg_intervalrelated(ecg_signals_s_l)/nk.ecg_intervalrelated(ecg_signals_b_l)
            processed_ecg_r=nk.ecg_intervalrelated(ecg_signals_s_r)/nk.ecg_intervalrelated(ecg_signals_b_r)
            # Average the Left and Right features
            processed_ecg=(processed_ecg_l+processed_ecg_r)/2
            # Fill in the ECG dataframe
            if not len(ECG):
                ECG=processed_ecg
            else:
                ECG=pd.concat([ECG,processed_ecg],ignore_index=True)
            print("\rProgress: %d%%" %(total/(23*18)*100),end="")

In [ ]:
# Save the features as a CSV file
ECG.to_csv("DREAMER_ECG.csv") 